In [1]:
import pandas as pd
from tqdm import tqdm
import os
import glob

In [2]:
def work_on_part(filename):
    df=pd.read_csv(filename)
    df['TIME_M']=df['TIME_M'].apply(lambda x : x[:-(len(x)-x.rfind(':'))])
#     print('sliced the time')
    df["DATETIME"] = df["DATE"].map(str).str.slice(0,10,1) + ' ' + df["TIME_M"].map(str)
#     print('created the datetime column')
    df["DATETIME"]=pd.to_datetime(df["DATETIME"])
#     print('converted to datetime')
    df.drop(['DATE','TIME_M'],1,inplace=True)
    agg_df=df.groupby(['DATETIME','SYM_ROOT']).sum()
#     print('grouped by sum')
    mean_df=df[['DATETIME','SYM_ROOT','PRICE']].groupby(['DATETIME','SYM_ROOT']).mean()
#     print('grouped by and averaged')
    agg_df=agg_df.merge(mean_df,on=['DATETIME','SYM_ROOT'],how='inner')
#     print('merged the dfs')
    agg_df.drop('PRICE_x',1,inplace=True)
    agg_df.rename(columns={'PRICE_y': 'PRICE'}, inplace=True)
    agg_df.reset_index(level=[0,1], inplace=True)
#     print('reseted the index')
    agg_df.to_csv(f'{filename[:-4]}_minute.csv')

In [2]:
def divide_to_parts(filename,num_parts):
    df=pd.read_csv(filename,index_col='DATE')
    df.sort_index(inplace=True)
    uni_dates=sorted(list(set(list(df.index))))
    length=len(uni_dates)
    size_of_part=length//num_parts
    for i in tqdm(range(num_parts)):
        if i != range(num_parts)[-1]:
            part_df=df.loc[(uni_dates[i*size_of_part]):(uni_dates[(i+1)*size_of_part])]
            part_df.to_csv(f'SPYDIA_{(uni_dates[i*size_of_part])}_{(uni_dates[(i+1)*size_of_part])}.csv')
#             print(f'{(uni_dates[i*size_of_part])}:{(uni_dates[(i+1)*size_of_part])}')
        else:
            part_df=df.loc[(uni_dates[i*size_of_part]):]
            part_df.to_csv(f'SPYDIA_{(uni_dates[i*size_of_part])}_{uni_dates[-1]}.csv')
#             print(f'{(uni_dates[i*size_of_part])}:{uni_dates[-1]}')

In [3]:
divide_to_parts('SPYDIA_04052018-03052019.csv',10)

C:\Users\natan\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [19:10<00:00, 114.13s/it]


In [14]:
path = 'C:\\Users\\natan\\Documents\\GitHub\\capstone_project\\financial_data'
os.chdir(path)
result = glob.glob('*.{}'.format('csv'))

In [19]:
for file in tqdm(result):
    work_on_part(file)

100%|██████████████████████████████████████████████████████████████████████████████| 111/111 [1:08:51<00:00, 23.60s/it]


In [20]:
path = 'C:\\Users\\natan\\Documents\\GitHub\\capstone_project\\financial_data'
os.chdir(path)
result = glob.glob('*.{}'.format('csv'))

In [24]:
final_df=pd.DataFrame()
for filename in tqdm(result):
    temp_df=pd.read_csv(filename)
    final_df=final_df.append(temp_df)

100%|████████████████████████████████████████████████████████████████████████████████| 110/110 [00:28<00:00,  2.24it/s]


In [26]:
final_df.drop('Unnamed: 0',1,inplace=True)

In [27]:
final_df.drop_duplicates(inplace=True)

In [32]:
final_df.to_csv('04052009-18122019-SPYDIA-minutedata.csv',index=False)